In [2]:
import pandas, os
pd = pandas

results_df = {}
results_df['CountryandRegionMap__c'] = pd.read_excel('CountryandRegionMap__c.xlsx',index_col=0)
results_df['CPA_Costing__c'] = pd.read_excel('CPA_Costing__c.xlsx',index_col=0)
results_df['Currency_Management2__c'] = pd.read_excel('Currency_Management2__c.xlsx',index_col=0)
results_df['Freight__c'] = pd.read_excel('Freight__c.xlsx',index_col=0)
results_df['Part__c'] = pd.read_excel('Part__c.xlsx',index_col=0)
results_df['Shipment_Order__c'] = pd.read_excel('Shipment_Order__c.xlsx',index_col=0)
results_df['Shipment_Order_Package__c'] = pd.read_excel('Shipment_Order_Package__c.xlsx',index_col=0)
results_df['Task'] = pd.read_excel('Task.xlsx',index_col=0)
results_df['Tax_Condition__c'] = pd.read_excel('Tax_Condition__c.xlsx',index_col=0)
results_df['Tax_Structure_Per_Country__c'] = pd.read_excel('Tax_Structure_Per_Country__c.xlsx',index_col=0)

In [3]:
def get_so_dict(results_df, so_lookup_value):
    so_df = results_df['Shipment_Order__c']
    so_dict = so_df[so_df['Lookup Value'] == so_lookup_value].T.to_dict()
    so_id = list(so_dict.keys())[0]
    return so_dict[so_id]

In [4]:
def get_li_dict(results_df, so_lookup_value):
    li_df = results_df['Part__c']
    return li_df[li_df['Shipment_Order__r.Client_Reference__c'] == so_lookup_value].T.to_dict()

In [5]:
def get_tax_structure_dict(results_df, clearance_country):
    tax_structures = results_df['Tax_Structure_Per_Country__c']
    return tax_structures[tax_structures['Country__r.Name'] == clearance_country].T.to_dict()

In [6]:
def get_tax_type(tax_structure_item):
    tax_type = tax_structure_item['Tax_Type__c']
    tax_rate = tax_structure_item['Rate__c']
    tax_amount = tax_structure_item['Amount__c']
    tax_part_specific = tax_structure_item['Part_Specific__c']
    if tax_type == 'VAT':
        return 'VAT'
    elif tax_type == 'Duties' and tax_part_specific == True:
        return 'Duties Part Specific'
    elif tax_type != 'Duties' and tax_part_specific == True:
        return 'Other Part Specific'
    elif pd.isna(tax_rate):
        return 'Amount'
    else:
        return 'Rate Non Part Specific'      

In [7]:
def get_tax_conditions_dict(results_df, tax_structure_item):
    tax_calculation_id = tax_structure_item['Id']
    tax_conditions = results_df['Tax_Condition__c']
    return tax_conditions[tax_conditions['Tax_Structure_Per_Country__c'] == tax_calculation_id].T.to_dict()

In [8]:
def get_tax_amount(tax_structure_item,so_tax_item,tax_type,li_tax_item={}):
    so_tax_amount = tax_structure_item['Amount__c']
    so_has_line_items = so_tax_item['of_Line_Items__c'] > 0
    number_of_line_items = so_tax_item['of_Line_Items__c']
    shipment_value = so_tax_item['Shipment_Value_USD__c']
    
    if so_has_line_items:
        li_value = li_tax_item['Total_Value__c']
        li_proportion = 1 / number_of_line_items
        #print(li_proportion)
        tax_amount = so_tax_amount * li_proportion
        #print(tax_amount)
        return tax_amount
    else:
        return so_tax_amount

In [9]:
def get_tax_rate(tax_structure_item,so_tax_item,tax_type,li_tax_item={}):
    tax_structure_item_rate = tax_structure_item['Rate__c']/100
    tax_structure_item_default_rate = tax_structure_item['Default_Duty_Rate__c']/100
    so_has_line_items = so_tax_item['of_Line_Items__c'] > 0
    if tax_type == 'VAT':
        if so_has_line_items:
            if pd.isna(li_tax_item['Specific_VAT_Rate__c']):
                tax_item_rate = tax_structure_item_rate
            else:
                tax_item_rate = li_tax_item['Specific_VAT_Rate__c']/100
        else:
            tax_item_rate = tax_structure_item_rate
    elif tax_type == 'Duties Part Specific':
        if so_has_line_items:
            if pd.isna(li_tax_item['Rate__c']):
                tax_item_rate = tax_structure_item_default_rate
            else:
                tax_item_rate = li_tax_item['Rate__c']/100
        else:
            tax_item_rate = tax_structure_item_default_rate
    elif tax_type == 'Other Part Specific':
        if so_has_line_items:
            if pd.isna(li_tax_item['Part_Specific_Rate_1__c']):
                tax_item_additional_rate_1 = 0
            else:
                tax_item_additional_rate_1 = li_tax_item['Part_Specific_Rate_1__c']/100
            if pd.isna(li_tax_item['Part_Specific_Rate_2__c']):
                tax_item_additional_rate_2 = 0
            else:
                tax_item_additional_rate_2 = li_tax_item['Part_Specific_Rate_2__c']/100
            if pd.isna(li_tax_item['Part_Specific_Rate_3__c']):
                tax_item_additional_rate_3 = 0
            else:
                tax_item_additional_rate_3 = li_tax_item['Part_Specific_Rate_3__c']/100
            tax_structure_item_additional = int(tax_structure_item['Additional_Part_Specific_Tax__c'])
            tax_item_rate = eval(f'tax_item_additional_rate_{tax_structure_item_additional}')
        else:
            tax_item_rate = 0
    elif tax_type == 'Rate Non Part Specific':
        tax_item_rate = tax_structure_item_rate
    return tax_item_rate

In [10]:
def get_cif_value(tax_structure_item,so_tax_item,li_tax_item={}):
    cif_absolute_value_adjustment = tax_structure_item['Country__r.CIF_Absolute_value_adjustment__c']
    cif_adjustment_factor = tax_structure_item['Country__r.CIF_Adjustment_Factor__c']/100
    
    cpa_cif = so_tax_item['CPA_v2_0__r.CIF_Freight_and_Insurance__c']
    client_actual_freight = so_tax_item['Client_Actual_Freight_Value__c']
    client_actual_insurance = so_tax_item['Client_Actual_Insurance_Value__c']
    so_has_line_items = so_tax_item['of_Line_Items__c'] > 0
    shipment_value = so_tax_item['Shipment_Value_USD__c']
    
    if pd.isna(cpa_cif):
        so_cif_value = (shipment_value + cif_absolute_value_adjustment) * (1 + cif_adjustment_factor)
    else:
        so_cif_value = shipment_value + client_actual_freight + client_actual_insurance
        
    if so_has_line_items:
        li_value = li_tax_item['Total_Value__c']
        li_proportion = li_value / shipment_value
        li_cif_value = so_cif_value * li_proportion
        return li_cif_value
    else:
        return so_cif_value

In [11]:
def get_fob_value(tax_structure_item,so_tax_item,li_tax_item={}):
    so_has_line_items = so_tax_item['of_Line_Items__c'] > 0
    so_fob_value = so_tax_item['Shipment_Value_USD__c']
    
    if so_has_line_items:
        li_value = li_tax_item['Total_Value__c']
        li_proportion = li_value / so_fob_value
        li_fob_value = so_fob_value * li_proportion
        return li_fob_value
    else:
        return so_fob_value

In [12]:
def get_previous_tax_total(tax_type,so_tax_item,so_tax_totals_dict,li_tax_totals_dict,order_number,lookup_value):
    of_line_items = so_tax_item['of_Line_Items__c']
    so_has_line_items = of_line_items > 0
    if so_has_line_items:
        if tax_type == 'Rate Non Part Specific' or tax_type == 'VAT':
            return so_tax_totals_dict[order_number] / of_line_items
        else:
            return li_tax_totals_dict[lookup_value][order_number]
    else:
        return so_tax_totals_dict[order_number]

In [13]:
def get_taxable_value(so_tax_totals_dict,li_tax_totals_dict,tax_structure_item,so_tax_item,tax_type,li_tax_item={}):
    applied_to_string = tax_structure_item['Applied_to_Value__c']
    order_number = tax_structure_item['Order_Number__c']
    additional_percent = tax_structure_item['Additional_Percent__c']
    additional_applies_to = tax_structure_item['Applies_to_Order__c']
    so_has_line_items = so_tax_item['of_Line_Items__c'] > 0
    so_lookup_value = so_tax_item['Lookup Value']
    
    if (tax_type == 'VAT' or tax_type == 'Duties Part Specific' or tax_type == 'Other Part Specific' or tax_type == 'Rate Non Part Specific'):
        if so_has_line_items:
            li_lookup_value = li_tax_item['Lookup Value']
            if applied_to_string == 'CIF':
                taxable_value = get_cif_value(tax_structure_item, so_tax_item, li_tax_item)
            elif applied_to_string == 'FOB':
                taxable_value = get_fob_value(tax_structure_item, so_tax_item, li_tax_item)
            else:
                taxable_value = get_previous_tax_total(tax_type, so_tax_item,so_tax_totals_dict,li_tax_totals_dict,applied_to_string,li_lookup_value)
            
        else:
            if applied_to_string == 'CIF':
                taxable_value = get_cif_value(tax_structure_item, so_tax_item)
            elif applied_to_string == 'FOB':
                taxable_value = get_fob_value(tax_structure_item, so_tax_item)
            else:
                taxable_value = get_previous_tax_total(tax_type, so_tax_item,so_tax_totals_dict,li_tax_totals_dict,applied_to_string,so_lookup_value)
                #print(taxable_value)
        
        if not pd.isna(additional_percent):
            taxable_value = taxable_value * (1 + additional_percent/100)
        
        if not pd.isna(additional_applies_to):
            for additional_applies_to_item in additional_applies_to.split(','):
                if so_has_line_items:
                    taxable_value += get_previous_tax_total(tax_type, so_tax_item,so_tax_totals_dict,li_tax_totals_dict,additional_applies_to_item,li_lookup_value)
                else:
                    taxable_value += get_previous_tax_total(tax_type, so_tax_item,so_tax_totals_dict,li_tax_totals_dict,additional_applies_to_item,so_lookup_value)
                    
    return taxable_value

In [14]:
def so_tax_totals_rollup(li_tax_totals_dict,so_tax_totals_dict):
    for tax_order in li_tax_totals_dict[list(li_tax_totals_dict.keys())[0]].keys():
        so_tax_totals_dict[tax_order] = 0
    
    for li in li_tax_totals_dict.keys():
        for tax_order in li_tax_totals_dict[li].keys():
            so_tax_totals_dict[tax_order] += li_tax_totals_dict[li][tax_order]
                    
    return so_tax_totals_dict

In [15]:
def apply_max_min(so_tax_totals_dict,tax_max,tax_min,tax_order_number):
    tax_total = so_tax_totals_dict[tax_order_number]
    
    if not pd.isna(tax_max):
        tax_total = min(tax_total,tax_max)
   
    if not pd.isna(tax_min):
        tax_total = max(tax_total,tax_min)

    so_tax_totals_dict[tax_order_number] = tax_total
    
    return so_tax_totals_dict

In [16]:
def check_tax_condition(so_record, tax_condition, currency_converter_df, EU_regions_df, tax_condition_numbers_bool_dict, li_record=None, tax_type=None):
    tax_condition_class = tax_condition['Condition_Value__c']
    tax_condition_condition = tax_condition['Condition__c']
    tax_condition_number = str(tax_condition['Condition_Number__c'])

    # 'CIF Tests' & 'Shipment Value Tests'
    if tax_condition_class == 'CIF Value' or tax_condition_class == 'Shipment Value':
        value_CIF = so_record['CIF_Value__c']
        value_Shipment = so_record['Shipment_Value_USD__c']
        value = value_CIF if (tax_condition_class == 'CIF') else value_Shipment
        condition_currency = tax_condition['Condition_Currency__c']
        
        if condition_currency != 'US Dollar (USD)':
            # convert value_Shipment to other currency
            conversion_rate = currency_converter_df[currency_converter_df['Name'] == condition_currency]['Conversion_Rate__c'][0]
            value = value/conversion_rate
        
        value_to_test_1 = tax_condition['Condition_Floor__c']
        if tax_condition_condition == '<':
            tax_condition_numbers_bool_dict[tax_condition_number] = str(value < value_to_test_1)
        elif tax_condition_condition == '<=':
            tax_condition_numbers_bool_dict[tax_condition_number] = str(value <= value_to_test_1)
        elif tax_condition_condition == '>':
            tax_condition_numbers_bool_dict[tax_condition_number] = str(value > value_to_test_1)
        elif tax_condition_condition == '>=':
            tax_condition_numbers_bool_dict[tax_condition_number] = str(value >= value_to_test_1)
        elif tax_condition_condition == 'Between':
            value_to_test_2 = tax_condition['Condition_Ceiling__c']
            tax_condition_numbers_bool_dict[tax_condition_number] = str(value > value_to_test_1 and value < value_to_test_2)

    # 'COO Tests' & 'Ship From Country Tests' 
    elif tax_condition_class == 'Ship From Country' or tax_condition_class == 'COO':
        ship_from_country = so_record['Ship_From_Country__c']
        if li_record is not None:
            coo_country = li_record['Country_of_Origin2__c']
        else:
            coo_country = 'NotProvided'
        country = coo_country if (tax_condition_class == 'COO') else ship_from_country
        regions_to_test = tax_condition['Condition_Region__c']
        # 'Catalogue_Part__r.COO_2_digit__c', 
        # 'COO_2_digit__c', 

        if regions_to_test == 'European Union':
            # sets country as EU if TRUE
            in_EU = so_record['Ship_From_Country__c'] in EU_regions_df['Country__c'].unique()
            if in_EU:
                 country = 'European Union'

        if tax_condition_condition == 'Does Not Contain':
            if country != regions_to_test:
                tax_condition_numbers_bool_dict[tax_condition_number] = 'True'
            else:
                tax_condition_numbers_bool_dict[tax_condition_number] = 'False'
        elif tax_condition_condition == 'Contains':
            if country == regions_to_test:
                tax_condition_numbers_bool_dict[tax_condition_number] = 'True'
            else:
                tax_condition_numbers_bool_dict[tax_condition_number] = 'False'
    
    else:
        print(f"Error, tax_condition_class incorrect GOT - {tax_condition_class}")       

    return tax_condition_numbers_bool_dict                 


In [17]:
def check_LI_tax(results_df, tax_record, so_record, tax_conditions_dict, logic_str, li_record=None, tax_type=None):
    final_ans = True
                    
    # if tax_record['Id'] == 'a1c1v000005Bt5oAAC':
    #     print('Here')
    
    if not pd.isna(logic_str):
        tax_condition_numbers_bool_dict = {}
        for tax_cond_iter in tax_conditions_dict.keys():
            # Cycles through each TAX CONDITION, gets the TAX CONDITION
            # then does the tests to see if they meet the conditions for the CONDITION

            # print(tax_conditions_dict.keys())   # checks how many tax conditions

            tax_condition = tax_conditions_dict[tax_cond_iter]
            tax_condition_numbers_bool_dict.update(check_tax_condition(so_record, 
                                                                    tax_condition,
                                                                    currency_converter_df=results_df['Currency_Management2__c'], 
                                                                    EU_regions_df=results_df['CountryandRegionMap__c'], 
                                                                    tax_condition_numbers_bool_dict=tax_condition_numbers_bool_dict,
                                                                    li_record=li_record, 
                                                                    tax_type=tax_type))    
            # print(tax_condition_numbers_bool_dict)  # checks the conditions dict                 

        logic = logic_str.lower().split(" ")
        
        built_str = ''
        for i in range(0, len(logic)):
            condition_key = logic[i]
            if condition_key in tax_condition_numbers_bool_dict.keys():
                built_str = built_str + tax_condition_numbers_bool_dict[logic[i]] + " "
            else:
                built_str = built_str + logic[i] + " "

        final_ans = eval(built_str)
        # print(f"\t HAS {len(tax_conditions_dict.keys())} CONDITIONS: [{logic_str}] | Tax Conditions Check: {tax_condition_numbers_bool_dict} | Final: {final_ans}")
    
    return final_ans
    

In [54]:
def total_taxes(results_df, lookup_value):
    so_tax_totals_dict = {}
    li_tax_totals_dict = {}
    so_record = get_so_dict(results_df, lookup_value)
    li_records = get_li_dict(results_df, lookup_value)
    clearance_country = so_record['CPA_v2_0__r.Final_Destination__c']
    tax_structure_dict = get_tax_structure_dict(results_df, clearance_country)
    so_has_line_items = so_record['of_Line_Items__c'] > 0
    
    for tax_record_key in tax_structure_dict.keys():
        tax_record = tax_structure_dict[tax_record_key]
        tax_order_number = str(tax_record['Order_Number__c'])
        tax_type = get_tax_type(tax_record)
        tax_max = tax_record['Max__c']
        tax_min = tax_record['Min__c']
        logic_str = tax_record['Condition_Logic__c']
        tax_conditions_dict = get_tax_conditions_dict(results_df, tax_record)
        if so_has_line_items:
            for li_record_key in li_records.keys():
                li_record = li_records[li_record_key]
                li_lookup_value = li_records[li_record_key]['Id']
                li_tax_totals_dict[li_lookup_value] = {}
        #____________________________________________________________________________________________________________________________________________________

                multiplicative_factor = 1
                
                final_ans = check_LI_tax(results_df, tax_record, so_record, tax_conditions_dict, logic_str, li_record=li_record, tax_type=None)

                if not final_ans:
                    multiplicative_factor = 0
        
        #____________________________________________________________________________________________________________________________________________________    
                if tax_type == 'Amount':
                    tax_amount = get_tax_amount(tax_record,so_record,tax_type,li_record)
                    #print(tax_amount)
                if tax_type != 'Amount':
                    tax_rate = get_tax_rate(tax_record,so_record,tax_type,li_record)
                    taxable_value = get_taxable_value(so_tax_totals_dict,li_tax_totals_dict,tax_record,so_record,tax_type,li_record)
                    #print(tax_rate)
                    #print(taxable_value)    
                #print(tax_structure_dict[tax_record_key]['Tax_Type__c'])
                if tax_type == 'Amount':
                    tax_total = tax_amount
                else:
                    tax_total = taxable_value * tax_rate
                #print(tax_total)
                #if tax_order_number == '12':
                #    print(li_lookup_value)
                #    print(tax_rate)
                #    print(taxable_value)
                #    print(tax_total)
                li_tax_totals_dict[li_lookup_value][tax_order_number] = tax_total*multiplicative_factor

                #print(li_tax_totals_dict)
            so_tax_totals_dict = so_tax_totals_rollup(li_tax_totals_dict,so_tax_totals_dict)
            so_tax_totals_dict = apply_max_min(so_tax_totals_dict,tax_max,tax_min,tax_order_number)
                           
        else:
        #____________________________________________________________________________________________________________________________________________________

            multiplicative_factor = 1
            
            final_ans = check_LI_tax(results_df, tax_record, so_record, tax_conditions_dict, logic_str, tax_type=None)

            if not final_ans:
                multiplicative_factor = 0
        
        #____________________________________________________________________________________________________________________________________________________   
            if tax_type == 'Amount':
                tax_amount = get_tax_amount(tax_record,so_record,tax_type)
                #print(tax_amount)
            if tax_type != 'Amount':
                tax_rate = get_tax_rate(tax_record,so_record,tax_type)
                taxable_value = get_taxable_value(so_tax_totals_dict,li_tax_totals_dict,tax_record,so_record,tax_type)
                #print(tax_rate)
                #print(taxable_value)
            #print(tax_structure_dict[tax_record_key]['Tax_Type__c'])
            if tax_type == 'Amount':
                tax_total = tax_amount
            else:
                tax_total = taxable_value * tax_rate
            #print(tax_total)
            so_tax_totals_dict[tax_order_number] = tax_total*multiplicative_factor
            so_tax_totals_dict = apply_max_min(so_tax_totals_dict,tax_max,tax_min,tax_order_number)
            
            # print(so_tax_totals_dict)
    # print(so_tax_totals_dict)
    # print(li_tax_totals_dict)
    total_tax_for_so = 0
    for order_key in so_tax_totals_dict.keys():
        total_tax_for_so += so_tax_totals_dict[order_key]
    return total_tax_for_so

In [55]:
total_taxes(results_df, 'SO04')

3609.92

In [57]:
for so in ['SO01','SO02','SO03','SO04','SO05','SO06','SO07','SO08','SO09','SO10','SO11','SO12','SO13','SO14','SO15','SO16','SO17','SO18','SO19','SO20','SO21','SO22','SO24','SO25','SO26']:
    out = total_taxes(results_df, so)
    print(f'{so}: {out}')

SO01: 0.5
SO02: 622.0605
SO03: 1612.25
SO04: 3609.92
SO05: 27090.0
SO06: 18232.575
SO07: 526.993068
SO08: 2.0
SO09: 10000.0
SO10: 209.35
SO11: 2785.4112000000005
SO12: 351.5
SO13: 1963.6228750500002
SO14: 313.11524999999995
SO15: 157.0
SO16: 120.91654126000002
SO17: 317.842
SO18: 538.125
SO19: 185.353
SO20: 2131.60233
SO21: 14380.595000000001
SO22: 0.0
SO24: 284471.32658500003
SO25: 255727.09329115634
SO26: 430.0


In [51]:
for i in range(24, 25):
    print(i)
    if i > 9:
        _filter = 'SO'+str(i)
    else:
        _filter = 'SO0'+str(i)
    total_so_tax, line_item_data = total_taxes(results_df, _filter)
    # print(f"{_filter}: {tt}")

24


In [49]:
total_so_tax_24, line_item_data_24 = total_taxes(results_df, 'SO24')
df_line_item_data_24 = pd.DataFrame(line_item_data_24)

total_so_tax_25, line_item_data_25 = total_taxes(results_df, 'SO25')
df_line_item_data_25 = pd.DataFrame(line_item_data_25)

with pd.ExcelWriter('Data.xlsx') as writer:
    df_line_item_data_24.to_excel(writer, sheet_name='Data 24')
    df_line_item_data_25.to_excel(writer, sheet_name='Data 25')

In [156]:
x = get_tax_structure_dict(results_df, 'United States')

In [157]:
x.keys()

dict_keys(['a1c0Y0000034h4xQAA', 'a1c0Y0000034h52QAA', 'a1c1v000005Bt5oAAC', 'a1c1v0000055mC0AAI', 'a1c07000000PDZKAA4'])

In [ ]:
x['a1c1v000005C958AAC']['Condition_Logic__c']

In [34]:
results_df['Shipment_Order__c'][results_df['Shipment_Order__c']['Lookup Value']=='SO10']

,Id,Shipment_Value_USD__c,Total_Cost_Range_Min__c,Liability_Cover_Fee_USD__c,FC_Total_License_Cost__c,Pre_inspection_Responsibility__c,Who_arranges_International_courier__c,Total_Invoice_Amount_Excl_Taxes_Duties__c,CIF_Value__c,FC_Total_Handling_Costs__c,...,IOR_Price_List__r.EOR_Clearance__c,IOR_Price_List__r.Estimated_Customs_Handling__c,IOR_Price_List__r.EOR_Licenses_permits__c,IOR_Price_List__r.Tax_Buffer__c,IOR_Price_List__r.Estimated_Customs_Clearance__c,Account__r.Client_type__c,Account__r.Staging_Warehouse_Preferred_Method__c,Account__r.Name,Account__r.Pre_Inspection_Responsibility_Preference__c,Lookup Value
a0R07000000PxyFEAS,a0R07000000PxyFEAS,6000.0,1749.31,90.0,0.0,NaN,TecEx,1749.31,6000.0,304.35,...,NaN,350.0,NaN,2,100.0,Distributor,Yes,Test team - NL No Account,NaN,SO10


In [22]:
results_df['CountryandRegionMap__c']

,Id,Country__c,Lookup Value
a421v000000eEklAAE,a421v000000eEklAAE,Ceuta,A421V000000EEKLAAE
a421v000000tx9YAAQ,a421v000000tx9YAAQ,Melilla,A421V000000TX9YAAQ
a421v0000001yDuAAI,a421v0000001yDuAAI,France,A421V0000001YDUAAI
a421v0000001yDyAAI,a421v0000001yDyAAI,Germany,A421V0000001YDYAAI
a421v0000001yE2AAI,a421v0000001yE2AAI,Greece,A421V0000001YE2AAI
a421v0000001yEBAAY,a421v0000001yEBAAY,Hungary,A421V0000001YEBAAY
a421v0000001yEHAAY,a421v0000001yEHAAY,Ireland,A421V0000001YEHAAY
a421v0000001yEKAAY,a421v0000001yEKAAY,Italy,A421V0000001YEKAAY
a421v0000001yEYAAY,a421v0000001yEYAAY,Latvia,A421V0000001YEYAAY
a421v0000001yEeAAI,a421v0000001yEeAAI,Lithuania,A421V0000001YEEAAI
